Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

# Installing Libraries

In [28]:
# !echo $CUDA_HOME
!export PATH="/usr/local/cuda/bin:$PATH"
# !nvidia-smi
# !cuda-toolkit --version

In [21]:
# For download the models
!pip install huggingface_hub -qU
!pip install datasets -qU

/usr/local/cuda


In [30]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir -qU
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir -q

  error: subprocess-exited-with-error
  
  × Building wheel for llama-cpp-python (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      *** scikit-build-core 0.9.2 using CMake 3.29.2 (wheel)
      *** Configuring CMake...
      2024-04-25 09:44:11,297 - scikit_build_core - WARNING - libdir/ldlibrary: /opt/conda/lib/libpython3.10.a is not a real file!
      2024-04-25 09:44:11,297 - scikit_build_core - WARNING - Can't find a Python library, got libdir=/opt/conda/lib, ldlibrary=libpython3.10.a, multiarch=x86_64-linux-gnu, masd=None
      loading initial cache file /tmp/tmpntnomri_/build/CMakeInit.txt
      -- The C compiler identification is GNU 9.4.0
      -- The CXX compiler identification is GNU 9.4.0
      -- Detecting C compiler ABI info
      -- Detecting C compiler ABI info - done
      -- Check for working C compiler: /usr/bin/cc - skipped
      -- Detecting C compile features
      -- Detecting C compile features - done
      -- Detecting CX

In [5]:
!pip install llama_cpp_python -qU

# Loading Model

In [8]:
from huggingface_hub import hf_hub_download

# model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
# model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"
model_name_or_path = "QuantFactory/Meta-Llama-3-8B-Instruct-GGUF"
model_basename = "Meta-Llama-3-8B-Instruct.Q2_K.gguf"
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [14]:
# This config has been tested on an RTX 3080 (VRAM of 16GB). You might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=16, # CPU cores
    n_batch=8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True,
    verbose=False
)

run_on_test = False # whether this baseline system is ran on the test splits or the val splits

# Running the model

In [11]:
import os
print(os.listdir("/kaggle/input/"))

if not os.path.exists('/kaggle/working/output/'):
    os.mkdir('/kaggle/working/output/')

In [15]:
MODEL_NAME = model_name_or_path.split('/')[1]
# make sure the output filename is the same as the reference filename for the scoring program
path_val_model_agnostic = "/kaggle/input/nlp-project-dataset/data/val/val.model-agnostic.json"
path_val_model_agnostic_output = f"/kaggle/working/output/{MODEL_NAME}/val.model-agnostic.json"

from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# alternatively, one can use HuggingFace's library to load the data
dataset = load_dataset('json', data_files={'val':path_val_model_agnostic})
data_val_all = dataset['val']
num_sample = len(data_val_all)
print(dataset)
print(num_sample)

output_json = []
labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Context: {}
Sentence: {}
Is the sentence supported by the context above?
Answer Yes or No:
"""
for i in tqdm.trange(num_sample):
    # label = str(data_val_all[i]['label'])
    task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['id'])
    hyp = str(data_val_all[i]['hyp'])
    src = str(data_val_all[i]['src'])
    tgt = str(data_val_all[i]['tgt'])

    if task == "PG":
        context = f"Context: {src}"
    else: #i.e. task == "MT" or task == "DM":
        context = f"Context: {tgt}"

    sentence = f"Sentence: {hyp}"
    message = f"{context}\n{sentence}\nIs the Sentence supported by the Context above? Answer using ONLY yes or no:"
    prompt = f"<s>[INST] {message} [/INST]"

    response = lcpp_llm(
        prompt=prompt,
        temperature= 0.0,
        logprobs=1,
    )
    answer = str(response["choices"][0]["text"]).strip().lower()
    if answer.startswith("yes"):
        output_label = "Not Hallucination"
        prob = 1-float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if answer.startswith("no"):
        output_label = "Hallucination"
        prob = float(np.exp(response["choices"][0]["logprobs"]["token_logprobs"][0]))
    if not answer.startswith("no") and not answer.startswith("yes"):
        idx_random = random.randint(0,len(labels)-1)
        output_label = labels[idx_random]
        prob = float(0.5)

    item_to_json = {"label":output_label, "p(Hallucination)":prob}
    if run_on_test:
        item_to_json['id'] = id
    output_json.append(item_to_json)


f = open(path_val_model_agnostic_output, 'w', encoding='utf-8')
json.dump(output_json, f)
f.close()
print("done")

DatasetDict({
    val: Dataset({
        features: ['hyp', 'p(Hallucination)', 'ref', 'labels', 'tgt', 'task', 'label', 'src', 'model'],
        num_rows: 499
    })
})
499


  0%|          | 0/499 [00:00<?, ?it/s]

KeyboardInterrupt: 